<a href="https://colab.research.google.com/github/kevinhincapie/CAPSTONE-PROYECT/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto_2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

/bin/bash: conda: command not found


In [2]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

/bin/bash: conda: command not found


In [3]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

/bin/bash: conda: command not found


In [0]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [0]:
soup = BeautifulSoup(data, 'html.parser')

In [0]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [0]:
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [0]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [9]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


In [0]:
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned\n"].reset_index(drop=True)
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))

In [0]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned\n":
        row["Neighborhood"] = row["Borough"]

In [0]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G\n", "M2H\n", "M4B\n", "M1J\n", "M4G\n", "M4M\n", "M1R\n", "M9V\n", "M9L\n", "M5V\n", "M1B\n", "M5A\n"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)

PRINTING NUMBER OF ROWS

In [13]:
toronto_df_grouped.shape

(103, 3)

READING CSV FILE THROUGH PANDA

In [14]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


COMBINING DATA


In [16]:
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B\n,Scarborough\n,Malvern / Rouge,NaN,NaN
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek,NaN,NaN
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill,NaN,NaN
3,M1G\n,Scarborough\n,Woburn,NaN,NaN
4,M1H\n,Scarborough\n,Cedarbrae,NaN,NaN


CHEKING COORDINATATES

In [17]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G\n", "M2H\n", "M4B\n", "M1J\n", "M4G\n", "M4M\n", "M1R\n", "M9V\n", "M9L\n", "M5V\n", "M1B\n", "M5A\n"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G\n,Downtown Toronto\n,Central Bay Street,NaN,NaN
1,M2H\n,North York\n,Hillcrest Village,NaN,NaN
2,M4B\n,East York\n,Parkview Hill / Woodbine Gardens,NaN,NaN
3,M1J\n,Scarborough\n,Scarborough Village,NaN,NaN
4,M4G\n,East York\n,Leaside,NaN,NaN
5,M4M\n,East Toronto\n,Studio District,NaN,NaN
6,M1R\n,Scarborough\n,Wexford / Maryvale,NaN,NaN
7,M9V\n,Etobicoke\n,South Steeles / Silverstone / Humbergate / Jam...,NaN,NaN
8,M9L\n,North York\n,Humber Summit,NaN,NaN
9,M5V\n,Downtown Toronto\n,CN Tower / King and Spadina / Railway Lands / ...,NaN,NaN


LATITUDE LONGITUDE

In [19]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


MAKING A MAP AND MARKERS

In [0]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

NEIGHTBORHOODS IN TORONTO


In [21]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto\n', 'Central Toronto\n', 'Downtown Toronto\n', 'West Toronto\n']

In [22]:
# create a new DataFrame with only boroughs that contain the word Toronto
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E\n,East Toronto\n,The Beaches,NaN,NaN
1,M4K\n,East Toronto\n,The Danforth West / Riverdale,NaN,NaN
2,M4L\n,East Toronto\n,India Bazaar / The Beaches West,NaN,NaN
3,M4M\n,East Toronto\n,Studio District,NaN,NaN
4,M4N\n,Central Toronto\n,Lawrence Park,NaN,NaN


In [0]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

FOURSQUARE CRENTENTIALS

In [24]:
CLIENT_ID = 'LW1K4Y4X4QSJYNLUARXBZEU0YOMSKP1VOA1JEGCNMIOWHM2V' 
CLIENT_SECRET = '5IUQVADEKSZO3AFWTVB4I55A2NPNLOWIGHHJDJXRB1ZFXOGK'  
VERSION = '20190425' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LW1K4Y4X4QSJYNLUARXBZEU0YOMSKP1VOA1JEGCNMIOWHM2V
CLIENT_SECRET:5IUQVADEKSZO3AFWTVB4I55A2NPNLOWIGHHJDJXRB1ZFXOGK


In [0]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], 
                                                  toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ&client_secret=OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

CONVERTING LIST TO NEW DATAFRAME


In [26]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2418, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Japango,43.655268,-79.385165,Sushi Restaurant
2,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Poke Guys,43.654895,-79.385052,Poke Place
3,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Rolltation,43.654918,-79.387424,Japanese Restaurant
4,M4E\n,East Toronto\n,The Beaches,NaN,NaN,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant


HOW MANY VALUES ARE RETURNED

In [27]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E\n,East Toronto\n,The Beaches,0,0,62,62,62,62
M4K\n,East Toronto\n,The Danforth West / Riverdale,0,0,62,62,62,62
M4L\n,East Toronto\n,India Bazaar / The Beaches West,0,0,62,62,62,62
M4M\n,East Toronto\n,Studio District,0,0,62,62,62,62
M4N\n,Central Toronto\n,Lawrence Park,0,0,62,62,62,62
M4P\n,Central Toronto\n,Davisville North,0,0,62,62,62,62
M4R\n,Central Toronto\n,North Toronto West,0,0,62,62,62,62
M4S\n,Central Toronto\n,Davisville,0,0,62,62,62,62
M4T\n,Central Toronto\n,Moore Park / Summerhill East,0,0,62,62,62,62


CHEKING AREAS

In [28]:
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2418, 49)


,PostalCode,Borough,Neighborhoods,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bar,Breakfast Spot,Bubble Tea Shop,Café,Chinese Restaurant,Coffee Shop,Concert Hall,Dessert Shop,Donut Shop,Food Court,French Restaurant,Gastropub,Gift Shop,Gym,Gym / Fitness Center,Hotel,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Neighborhood,Office,Park,Plaza,Poke Place,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Smoke Shop,Sushi Restaurant,Tapas Restaurant,Tea Room,Toy / Game Store,University,Vegetarian / Vegan Restaurant
0,M4E\n,East Toronto\n,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M4E\n,East Toronto\n,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,M4E\n,East Toronto\n,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M4E\n,East Toronto\n,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M4E\n,East Toronto\n,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [29]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 49)


,PostalCode,Borough,Neighborhoods,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bar,Breakfast Spot,Bubble Tea Shop,Café,Chinese Restaurant,Coffee Shop,Concert Hall,Dessert Shop,Donut Shop,Food Court,French Restaurant,Gastropub,Gift Shop,Gym,Gym / Fitness Center,Hotel,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Neighborhood,Office,Park,Plaza,Poke Place,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Smoke Shop,Sushi Restaurant,Tapas Restaurant,Tea Room,Toy / Game Store,University,Vegetarian / Vegan Restaurant
0,M4E\n,East Toronto\n,The Beaches,0.016129,0.048387,0.016129,0.016129,0.032258,0.032258,0.016129,0.048387,0.016129,0.048387,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.064516,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.032258,0.016129,0.016129,0.032258
1,M4K\n,East Toronto\n,The Danforth West / Riverdale,0.016129,0.048387,0.016129,0.016129,0.032258,0.032258,0.016129,0.048387,0.016129,0.048387,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.064516,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.032258,0.016129,0.016129,0.032258
2,M4L\n,East Toronto\n,India Bazaar / The Beaches West,0.016129,0.048387,0.016129,0.016129,0.032258,0.032258,0.016129,0.048387,0.016129,0.048387,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.064516,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.032258,0.016129,0.016129,0.032258
3,M4M\n,East Toronto\n,Studio District,0.016129,0.048387,0.016129,0.016129,0.032258,0.032258,0.016129,0.048387,0.016129,0.048387,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.064516,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.032258,0.016129,0.016129,0.032258
4,M4N\n,Central Toronto\n,Lawrence Park,0.016129,0.048387,0.016129,0.016129,0.032258,0.032258,0.016129,0.048387,0.016129,0.048387,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.064516,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.032258,0.016129,0.016129,0.032258
5,M4P\n,Central Toronto\n,Davisville North,0.016129,0.048387,0.016129,0.016129,0.032258,0.032258,0.016129,0.048387,0.016129,0.048387,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.064516,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.032258,0.016129,0.016129,0.032258
6,M4R\n,Central Toronto\n,North Toronto West,0.016129,0.048387,0.016129,0.016129,0.032258,0.032258,0.016129,0.048387,0.016129,0.048387,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.064516,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.016129,0.016129,0.016129,0.016129,0.032258,0.016129,0.032258,0.016129,0.016129,0.032258
7,M4S\n,Central Toronto\n,Davisville,0.016129,0.048387,0.016129,0.016129,0.032258,0.032258,0.016129,0.048387,0.016129,

new data frame

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E\n,East Toronto\n,The Beaches,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
1,M4K\n,East Toronto\n,The Danforth West / Riverdale,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
2,M4L\n,East Toronto\n,India Bazaar / The Beaches West,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
3,M4M\n,East Toronto\n,Studio District,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
4,M4N\n,Central Toronto\n,Lawrence Park,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
5,M4P\n,Central Toronto\n,Davisville North,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
6,M4R\n,Central Toronto\n,North Toronto West,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
7,M4S\n,Central Toronto\n,Davisville,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
8,M4T\n,Central Toronto\n,Moore Park / Summerhill East,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
9,M4V\n,Central Toronto\n,Summerhill West / Rathnelly / South Hill / For...,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant


klustering

In [31]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  """


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [32]:
toronto_merged = toronto_df_new.copy()

toronto_merged["Cluster Labels"] = kmeans.labels_

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head()

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E\n,East Toronto\n,The Beaches,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
1,M4K\n,East Toronto\n,The Danforth West / Riverdale,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
2,M4L\n,East Toronto\n,India Bazaar / The Beaches West,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
3,M4M\n,East Toronto\n,Studio District,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
4,M4N\n,Central Toronto\n,Lawrence Park,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant


In [33]:
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E\n,East Toronto\n,The Beaches,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
21,M5L\n,Downtown Toronto\n,Commerce Court / Victoria Hotel,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
22,M5N\n,Central Toronto\n,Roselawn,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
23,M5P\n,Central Toronto\n,Forest Hill North & West,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
24,M5R\n,Central Toronto\n,The Annex / North Midtown / Yorkville,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
25,M5S\n,Downtown Toronto\n,University of Toronto / Harbord,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
26,M5T\n,Downtown Toronto\n,Kensington Market / Chinatown / Grange Park,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
27,M5V\n,Downtown Toronto\n,CN Tower / King and Spadina / Railway Lands / ...,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
20,M5K\n,Downtown Toronto\n,Toronto Dominion Centre / Design Exchange,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
28,M5W\n,Downtown Toronto\n,Stn A PO Boxes,NaN,NaN,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant


VISULIZATIONG CLUSTERS


In [0]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## CHECK CLUSTERS

1ST CLUSTER

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto\n,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
21,Downtown Toronto\n,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
22,Central Toronto\n,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
23,Central Toronto\n,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
24,Central Toronto\n,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
25,Downtown Toronto\n,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
26,Downtown Toronto\n,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
27,Downtown Toronto\n,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
20,Downtown Toronto\n,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant
28,Downtown Toronto\n,0,Japanese Restaurant,Art Gallery,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Ramen Restaurant,Hotel,Bar,Sushi Restaurant


2ND CLUSTER

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


3RD CLUSTER

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


4RD CLUSTER

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


5TH CLUSTER

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


# SUMMARY

You can find most of the clusters in cluster 1 where you can find most of the services you need, such as cafes, supermarkets and others.